In [ ]:
import control
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from dare.utils.nodeconstructorcable import NodeConstructorCable

In [ ]:
def generate_sin(num, t, V_eff=230 * np.sqrt(2), f0 = 50):
    v = list()
    for i in range(num):
        v.append(V_eff * np.sin(2*np.pi * f0 * t+i))
    out = np.vstack(v)
    return out

In [ ]:
# set up CM

CM = np.array([[0, 0, 1],
               [0, 0, 2],
               [-1, -2, 0]])

In [ ]:
Grid_S2_L1_2C = NodeConstructorCable(2, 1, CM = CM)

In [ ]:
A, B, C, D = Grid_S2_L1_2C.get_sys()

In [ ]:
sys = control.ss(A, B, C, D)

In [ ]:
# define time vector
ts = 1e-4
t_end = 0.005
steps = int(1/ts)
t = np.arange(0, t_end+ts, ts)
num_samples = len(t)

# generate init state
x0 = np.zeros((A.shape[0],1))

# simple input signal of constant 230V from all sources
u = np.array([230]).repeat(Grid_S2_L1_2C.num_source)[:,None] * np.ones((Grid_S2_L1_2C.num_source,len(t)))

T, yout, xout = control.forced_response(sys, T=t, U=u, X0=x0, return_x=True, squeeze=True)

plt.plot(t, xout[1], label='$v_1$')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
plt.title('Plot current $v_1$')
plt.legend()
plt.grid()
plt.show()